In [91]:
from road_segments import *
from os import listdir
from os.path import isfile, join

%load_ext autoreload
%autoreload 2


reloaded
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
request_url = build_url(gps_raw[:100])
result = requests.get(request_url, params=params)
response = result.json()

street_times = get_street_times(response)

street_points = get_street_points(street_times)
street_frame_bounds = get_street_bounds(street_times)

method      = o3d.registration.GlobalOptimizationLevenbergMarquardt()
criteria    = o3d.registration.GlobalOptimizationConvergenceCriteria()
option      = o3d.registration.GlobalOptimizationOption(
            max_correspondence_distance=0.05,
            edge_prune_threshold=0.25,
            preference_loop_closure=0.01,
            reference_node=0)


pg = o3d.io.read_pose_graph(dataset + "/pose/full.json")

g2n = get_gps_to_node(pg, gps_raw, 0)

#fullpg_to_segment['street'][segment][index] = original
fullpg_idxmap = {}

street_g2pg = {}
import os
for street, bounds in street_frame_bounds.items():
    print("Working on {}".format(street))
    
    npgs, g2pg, idxmap = chunk_pg(pg, g2n, int(bounds['start']), int(bounds['stop']))
    
    street_g2pg[street] = g2pg
    fullpg_idxmap[street] = idxmap
    print("Got {} pose graphs".format(len(npgs)))
    
    
    for idx in range(len(npgs)):
        streetdir = dataset + "/pose/" + street.replace(" ", "_")
        
        if not os.path.isdir(streetdir):
            os.mkdir(streetdir)
            
        o3d.registration.global_optimization(road, method, criteria, option)

        o3d.io.write_pose_graph(streetdir + "/{0:03}.json".format(idx), npgs[idx])
    


reloaded
Working on Rigi Avenue
hi
gstartstop 0 2
asdf
gstartstop 3 5
asdf
gstartstop 6 8
asdf
gstartstop 9 11
asdf
gstartstop 12 14
asdf
gstartstop 15 17
asdf
gstartstop 18 20
asdf
gstartstop 21 23
asdf
gstartstop 24 26
asdf
gstartstop 27 29
asdf
gstartstop 30 32
asdf
gstartstop 33 35
asdf
gstartstop 36 38
asdf
gstartstop 39 41
gstartstop 42 44
gstartstop 45 47
gstartstop 48 50
gstartstop 51 53
gstartstop 54 56
gstartstop 57 59
gstartstop 60 62
gstartstop 63 65
gstartstop 66 68
gstartstop 69 71
gstartstop 72 74
gstartstop 75 77
gstartstop 78 80
gstartstop 81 83
gstartstop 84 86
gstartstop 87 89
gstartstop 90 92
gstartstop 93 95
gstartstop 96 98
gstartstop 99 101
gstartstop 102 104
gstartstop 105 107
gstartstop 108 110
gstartstop 111 113
gstartstop 114 116
gstartstop 117 119
gstartstop 120 122
gstartstop 123 125
gstartstop 126 128
gstartstop 129 131
gstartstop 132 134
gstartstop 135 137
gstartstop 138 140
gstartstop 141 143
gstartstop 144 146
gstartstop 147 149
Got 13 pose graphs
Worki

In [100]:

method      = o3d.registration.GlobalOptimizationLevenbergMarquardt()
criteria    = o3d.registration.GlobalOptimizationConvergenceCriteria()
option      = o3d.registration.GlobalOptimizationOption(
            max_correspondence_distance=0.05,
            edge_prune_threshold=0.25,
            preference_loop_closure=0.01,
            reference_node=0)

street = 'Rigi Avenue'

streetdir = dataset + "/pose/" + street.replace(" ", "_")


for fidx in range(len(street_g2pg[street])):
        
    if fidx>5:
        break
        
    print("reading: {}".format(fidx))
    road = o3d.io.read_pose_graph(streetdir + "/{0:03}.json".format(fidx))
        
    sp = street_g2pg[street][fidx]
    
    print(len(road.nodes))
    print(len(sp))
    gps_start = np.array([sp[0][4],sp[0][5],1])
    gps_stop = np.array([sp[-1][4],sp[-1][5],1])
    gps_bearing = gps_stop - gps_start
    print(gps_start, gps_stop)
    print(np.linalg.norm(gps_bearing))
    print(gps_bearing)
    
    road_start = road.nodes[0].pose[:3,3]
    road_stop = road.nodes[len(road.nodes)-1].pose[:3,3]
    road_bearing = road_stop - road_start
    print(np.linalg.norm(road_bearing))
    print(road_bearing)
    
    #camera travels in Z direction, need to rotate it to align with gps bearing
    t = R.from_dcm(get_rotation(gps_bearing, road_bearing))
    
    print(t.apply(gps_bearing))

    #leftover disparity is dominated by pitch of camera 
    #camera_pitch = zrot.apply(gps_bearing)
    #pitch_rot = R.from_dcm(get_rotation(camera_pitch, road_bearing))

    #t = R.from_dcm(np.identity(3))
    
    gls = lineset_from_gps(sp, R.from_dcm(np.identity(3)))
    print(len(gls.points))
    pls = lineset_from_nodes(road)
    
    i = np.identity(4)
    i[:3,:3] = t.as_dcm()
    
    gls.transform(i)
    o3d.visualization.draw_geometries([gls, pls])
    
    
    
#     inject_gps_edges(road, sp, fullpg_idxmap[street][fidx][0], t)
#     o3d.io.write_pose_graph(streetdir + "/{0:03}_test.json".format(fidx), road)
    
#     o3d.registration.global_optimization(road, method, criteria, option)
#     o3d.io.write_pose_graph(streetdir + "/{0:03}_optimized_with_gps.json".format(fidx), road)


reading: 0
89
2
[4.10102287e+05 4.76929656e+06 1.00000000e+00] [4.10102287e+05 4.76929658e+06 1.00000000e+00]
0.01851012041995359
[0.00024362 0.01850852 0.        ]
0.28254222719171806
[-0.0387898  -0.03886497  0.27715515]
[-0.00254123 -0.00254615  0.0181572 ]
2
reading: 1
119
2
[4.10102282e+05 4.76929719e+06 1.00000000e+00] [4.10102178e+05 4.76929858e+06 1.00000000e+00]
1.3936211694416796
[-0.10385175  1.3897463   0.        ]
13.687066103120701
[-3.40497732 -2.14513509 13.08206037]
[-0.34669581 -0.2184183   1.3320193 ]
2
reading: 2
120
2
[4.10101508e+05 4.76930438e+06 1.00000000e+00] [4.10100978e+05 4.76930848e+06 1.00000000e+00]
4.13214614818426
[-0.52963599  4.09806265  0.        ]
17.978141765405717
[-1.62853557 -3.32566392 17.59265224]
[-0.37430715 -0.76437985  4.04354416]
2
reading: 3
119
2
[4.10099804e+05 4.76931823e+06 1.00000000e+00] [4.10099240e+05 4.76932389e+06 1.00000000e+00]
5.681500961065706
[-0.56344553  5.65349293  0.        ]
17.199354806850238
[-3.01663679 -1.8761186

In [86]:
import utm